Projet kaggle : Tweet Sentiment Extraction

Emma BEUZE, Nacim BOUIA, Antoine FRANCESCHI, Ayoub RHIM, Guillaume RUSSIAS

In [1]:
import matplotlib.pyplot as plt
import pandas
import os 

Il y a 4 types de colonnes :


*   textID - unique ID for each piece of text 
*   text - the text of the tweet 
*   sentiment - the general sentiment of the tweet        => il y a 3 sentiments : positive, neutral, negative
*   selected_text - (train only) the text that supports the tweet's sentiment 


**OBJECTIF :  examiner le sentiment donné pour un tweet donné et déterminer le mot ou la phrase qui le soutient le mieux.**






Page qui résume comment utiliser les pandas : https://ledatascientist.com/manipulez-vos-donnees-avec-pandas/

In [6]:
os.chdir(str(os.getcwd()+"\donnees"))

In [7]:
# Load the dataset
#train_set = pandas.read_csv(r"train.csv")

train_set = pandas.read_csv(r"train.csv")
test_set = pandas.read_csv(r"test.csv")

#print(train_set.shape)    => (27481, 4)  (textID, text, selected_text, sentiment )
#print(test_set.shape)     => (3534, 3)   (textID, text, sentiment)

train_data_text = train_set["text"]
train_data_sentiment = train_set["sentiment"]

N_train = train_data_text.shape[0]

train_data = [(train_data_text[k],train_data_sentiment[k]) for k in range(N_train)]
train_labels = train_set["selected_text"]


# check data makes sense

print("Exemple 1 :",train_data[1], "extrait choisi :", train_labels[1])
print("Exemple 2 :",train_data[59], "extrait choisi :", train_labels[59])
print("Exemple 3 :",train_data[750], "extrait choisi :", train_labels[750])

Exemple 1 : (' Sooo SAD I will miss you here in San Diego!!!', 'negative') extrait choisi : Sooo SAD
Exemple 2 : (' what fun are you speaking of?', 'neutral') extrait choisi : what fun are you speaking of?
Exemple 3 : (' Thanks! Welcome back!', 'positive') extrait choisi : Thanks! Welcome back!


In [8]:
train_set

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


Dans le concours, la métrique à utiliser est le score de Jaccard :

https://en.wikipedia.org/wiki/Jaccard_index 

https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50.



Dans un premier, voici la méthode que nous utilisons :
* On crée trois tableaux T+, T-, T~ qui contiennent les text et selected_text qui ont un sentiment respectivement positifs, négatifs et neutres.
* On s'inspire de la méthode kNN : pour un texte donné, on regarde quel est son sentiment. Ensuite on trouve quels sont ses k plus proches voisins dans le tableau T associé, en utilisant la mesure de Jaccard.
* Parmi ces textes voisins, on regarde mot à mot quels sont les mots avec la plus grande distance de Jaccard, et on choisit de sélectionner ces mots

In [5]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [8]:
str1 = "je m'appelle Emma blablabla"
str2 = "je m'appelle Emma blablabla"
jaccard(str1,str2)

1.0